In [ ]:
"""
Γεμίζω τον πίνακα με τις μετρήσεις
του labevent 8ώρου.

Ο πίνακας μου απαρτίζεται από 388 στήλες και 68.191 γραμμές,
οπότε μου δημιουργούν 26.458.108 κελιά.

Οι τιμές μου ανέρχονται στα 1.276.856 οπότε με μια αφαίρεση
προκείπτουν 25.181.252 NaN κελιά, πράγμα που επιβεβαιώνει
και η εντολή df_lab_tbl.isna().sum().sum()

"""

In [2]:
import pandas as pd

In [3]:
df_lab_tbl = pd.read_csv('D:\Thesis\exports\o17_empty_lab_tbl.csv', index_col = False)
df_data_labevents = pd.read_csv('D:\Thesis\mimic-iv-1.0\hosp\d_labitems.csv.gz')
df_mean_lab = pd.read_csv('D:\Thesis\exports\o16_labevents_grouped_8hours_date_time.csv')

In [4]:
# Φέρνω τις τελευταίες στήλες, πρώτες
for i in range (1, 5):
    cols = list(df_lab_tbl.columns)
    cols = [cols[-1]] + cols[:-1]
    df_lab_tbl = df_lab_tbl[cols]

In [5]:
# Ελέγχω αν υπάρχει NaN στην στήλη με τις μέσες τιμές, το αποτέλεσμα ισούται με μηδέν.
df_mean_lab['Mean_Lab'].isna().sum()

0

In [6]:
"""
Δημιουργώ εκ νέου την στήλη concat χωρίς
κενά ανάμεσα στα στοιχεία. Θα χρησιμοποιήσω
αυτή τη στήλη σαν index και τα κενά μπορεί
να δημιουργήσουν πρόβλημα.
"""
df_lab_tbl["concat"] = (df_lab_tbl["subject_id"].astype(str)
                          + df_lab_tbl["hadm_id"].astype(str)
                          + df_lab_tbl["date"].astype(str)
                          + df_lab_tbl["time"].astype(str))

# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat
df_lab_tbl['concat'] = df_lab_tbl['concat'].str.replace('-','')
df_lab_tbl['concat'] = df_lab_tbl['concat'].str.replace(':','')

# Ορίζω την στήλη concat ως index των γραμμών
df_lab_tbl = df_lab_tbl.set_index('concat')

In [7]:
"""
Δημιουργώ την στήλη concat και στον πίνακα
που περιλαμβάνει τις μετρήσεις για να είναι
το κοινό σημείο ανάμεσα στους δυο πίνακες.
"""
df_mean_lab["concat"] = (df_mean_lab["subject_id"].astype(str)
                         + df_mean_lab["hadm_id"].astype(str)
                         + df_mean_lab["date"].astype(str)
                         + df_mean_lab["time"].astype(str))

# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat
df_mean_lab['concat'] = df_mean_lab['concat'].str.replace('-','')
df_mean_lab['concat'] = df_mean_lab['concat'].str.replace(':','')

In [8]:
for row in df_mean_lab.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Mean_Lab
    df_lab_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [9]:
# Έλεγχος NaN περιοχών σε όλο το dataframe
df_lab_tbl.isna().sum().sum()

25181252

In [10]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_lab_tbl.to_csv('D:\Thesis\exports\o18_fill_lab_itemid_header_tbl.csv', index=False)

## Αντικατάσταση των κεφαλίδων

In [11]:
df_data_labevents = df_data_labevents.set_index('itemid')

In [12]:
# Ενοποίηση τιμών σε κελιά
df_data_labevents["header"] = (df_data_labevents["label"].astype(str)
                                                + " ("
                                                + df_data_labevents["fluid"].astype(str)
                                                + ")")

In [13]:
"""
Ελέγχω τις κεφαλίδες και αν δεν ανήκουν στις
τέσσερις πρώτες τιμές κάνω τις αντικαταστάσεις.
"""
for col in df_lab_tbl.columns:
    if ((col != 'subject_id')
        and (col != 'hadm_id')
        and (col != 'date')
        and (col != 'time')):
        temp = (df_data_labevents.loc[int(col), 'header'])
        df_lab_tbl.rename(columns={'{}'.format(col): '{}'.format(temp)}, inplace=True)

In [14]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τα label των παρατηρήσεων
df_lab_tbl.to_csv('D:\Thesis\exports\o19_fill_lab_label_header_tbl.csv', index=False)